# WepSIM (https://wepsim.github.io/wepsim/)<br> <u>Tutorial for the basic arithmetic instructions in RISC-V</u>

* **Please run next code cell first to start tutorial**


In [33]:
# @title { vertical-output: true, display-mode: "form" }
#
# Copyright 2015-2024 Felix Garcia Carballeira, Alejandro Calderon Mateos, Javier Prieto Cepeda, Saul Alonso Monsalve, Juan Banga Pardo
#
# This file is part of WepSIM.
#
# WepSIM is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# WepSIM is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with WepSIM.  If not, see <http://www.gnu.org/licenses/>.
#
!echo "(1/3) Installing pre-requisites..."
!npm install terser jq jshint yargs@16.2.0 clear inquirer@8.2.6 fuzzy inquirer-command-prompt inquirer-autocomplete-prompt@1 >& /dev/null
!pip install lzstring
!echo "(2/3) Downloading WepSIM..."
![ ! -f wepsim-2.3.7.zip ] && wget https://github.com/acaldero/wepsim/releases/download/v2.3.7/wepsim-2.3.7.zip >& /dev/null
!rm -fr   wepsim
!unzip -o wepsim-2.3.7.zip  >& /dev/null
!mv wepsim-2.3.7 wepsim
!echo "(3/3) Done!"

from pathlib import Path
import pandas as pd
import io
import lzstring
from IPython.display import IFrame
from google.colab import data_table

def show_results(filename):
  df1    = ''
  status = 0
  try:
    df1 = pd.read_csv(filename)
    df1.columns = df1.columns.str.strip()
    for item in df1.columns[:]:
        df1[item] = df1[item].replace("\t","",    regex=True)
        df1[item] = df1[item].replace("&nbsp;","", regex=True)
    rdf = data_table.DataTable(df1, include_index=False, num_rows_per_page=20)
    display(rdf)
    status = 1
  except:
    df1 = Path(filename).read_text()
    print(df1)
    status = 0
  return status, df1

def get_lz(filename):
  try:
    with open(filename,'r',encoding='utf-8') as f:
        cell_str = f.readlines()
        cell_str = ''.join(cell_str)
    x = lzstring.LZString()
    cell_lz = x.compressToBase64(cell_str)
  except:
    cell_lz = ''
  return cell_lz

def get_ws_url(asm, cpu):
  url = 'https://wepsim.github.io/wepsim/ws_dist/?'
  url = url + 'mode=ep&'
  url = url + 'notify=false&'
  url = url + 'simulator=assembly:simulator&'
  url = url + 'examples_set=' + cpu + '&'
  url = url + 'example=11&'
  url = url + 'asm=' + asm
  return url

def show_asm_in_ws(filename, cpu):
  try:
    lzasm = get_lz(filename)
    url   = get_ws_url(lzasm, cpu)
    display(IFrame(src=url, width="100%", height=700))
    status = 1
  except:
    status = 0
  return status


# Following code thanks to Rosa Filgueira Vicente
# (https://colab.research.google.com/github/rosafilgueira/Workflows_Seminar)
from google.colab import _message

def write_cell_above_to_file(search_term, filename):
  cell = get_cell_above(search_term)
  code_block = get_cell_code_block(cell['source'])
  with open(filename, 'w') as fp:
    fp.writelines(code_block)

def get_cell_above(search_term):
  nb = _message.blocking_request('get_ipynb')
  for i, cell in enumerate(nb['ipynb']['cells']):
    if search_term in ''.join(cell['source']):
      return nb['ipynb']['cells'][i - 1]

def get_cell_code_block(cell_lines):
  code_block = []
  in_block = False
  for ln in cell_lines:
    if '```' in ln:
       in_block = not in_block
       continue
    if in_block:
       code_block.append(ln)
  return code_block


(1/3) Installing pre-requisites...
(2/3) Downloading WepSIM...
(3/3) Done!


# <u>Load and copy instructions</u>

To test arithmetic instructions we are going to need numbers to operate with.<br>
Examples for arithmetic instructions start by loading two numbers in two registers, then the arithmetic instruction is apply to this pair of numbers.

So, how can we have the numbers in the registers?


## Load immediate: li

* The "li" instruction is used to load an immediate value into a register.

* Example:
  ```python
  .text
  main:
      li      a0, 3
      li      a1, 2
  ```


In [38]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:li1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep \
!                   -f ./wepsim/repo/microcode/rv32/ep_base.mc \
!                   -s /tmp/example.asm > /tmp/result.csv

show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 3,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x3; register PC = 0x8008
2,pc = 0x8008,li a1 2,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x2; register PC = 0x8010


## Copy value: mv

* The "mv" instruction is used to copy the immediate value stored in a register into another register.

* Example:
  ```python
  .text
  main:
      li    a0, 3
      li    a1, 2
      mv    a0, a1
  ```


In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:move1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep \
!                   -f ./wepsim/repo/microcode/rv32/ep_base.mc \
!                   -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 3,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x3; register PC = 0x8008
2,pc = 0x8008,li a1 2,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x2; register PC = 0x8010
4,pc = 0x8010,mv a0 a1,register R10 = 0x2; register PC = 0x8014


# <u>Arithmetic instructions</u>

In general, binaries arithmetic instructions take two numbers and apply some mathematical arithmetic to this two numbers, saving the result in a CPU register. For unaries arithmetic instructions is similar but only take one number to work with.

## Add two integers: add, addi

* "add" adds the content of two CPU registers (could be the same CPU register twice) and save the result into a CPU register (could be the same as either operands). The content is going to be interpreted as signed integers.
* "addi" adds the content of one CPU register with one immediate value, saving the result in a CPU register (could be the same as the operand).

* Example:
  ```python
  .text
  main:
      li      a0, 3
      li      a1, 2
      add     a2, a0, a1   # a2 = 3 + 2
      addi    a2, a0, 10   # a2 = 3 + 10
  ```


In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:addaddi1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 3,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x3; register PC = 0x8008
2,pc = 0x8008,li a1 2,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x2; register PC = 0x8010
4,pc = 0x8010,add a2 a0 a1,register R12 = 0x5; register PC = 0x8014
5,pc = 0x8014,addi a2 a0 10,register R12 = 0xd; register PC = 0x8018


## Subtract: sub

* "sub" substracts the content of two CPU registers (could be the same CPU register twice) and save the result into a CPU register (could be the same as either operands). The content is going to be interpreted as signed integers.

* Example:
  ```
  .text
  main:
    li      a0, 0
    li      a1, 2
    sub     a3, a1, a0   # a2 = 0 - 2
    sub     a2, a0, a1   # a2 = 0 - 2
  ```


In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:sub1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 0,register PC = 0x8004
1,pc = 0x8004,,register PC = 0x8008; register SR = 0x10000000
2,pc = 0x8008,li a1 2,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x2; register PC = 0x8010
4,pc = 0x8010,sub a3 a1 a0,register R13 = 0x2; register PC = 0x8014
5,pc = 0x8014,sub a2 a0 a1,register R12 = 0xfffffffe; register PC = 0x8018


## Multiply: mul

* "mul" multiplies the content of two CPU registers (could be the same CPU register twice) and save the result into a CPU register (could be the same as either operands). The content is going to be interpreted as signed integers.
  * Be careful with possible overflow: the result could be outside the representation rage used by the CPU.

* Example:
  ```
  .text
  main:
    li      a0, 5
    li      a1, 2
    mul     a2, a0, a1   # a2 = 5 * 2
    li      a0, 0x10000000
    li      a1, 0x10000000
    mul     a3, a0, a1   # a2 = ...
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:mul1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 5,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x5; register PC = 0x8008
2,pc = 0x8008,li a1 2,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x2; register PC = 0x8010
4,pc = 0x8010,mul a2 a0 a1,register R12 = 0xa; register PC = 0x8014
5,pc = 0x8014,li a0 0x10000000,register R10 = 0x10000000; register PC = 0x8018
6,pc = 0x8018,,register PC = 0x801c
7,pc = 0x801c,li a1 0x10000000,register R11 = 0x10000000; register PC = 0x8020
8,pc = 0x8020,,register PC = 0x8024
9,pc = 0x8024,mul a3 a0 a1,register PC = 0x8028


## Divide Signed: div

* "div" divides the content of two CPU registers (could be the same CPU register twice) and save the result into a CPU register (could be the same as either operands). The content is going to be interpreted as signed integers.
  * Be careful with possible exception: divide by zero.

* Example:
  ```
  .text
  main:
    li      a0, 6
    li      a1, 2
    div     a2, a0, a1   # a2 = 6 / 2
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:div1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 6,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x6; register PC = 0x8008
2,pc = 0x8008,li a1 2,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x2; register PC = 0x8010
4,pc = 0x8010,div a2 a0 a1,register R12 = 0x3; register PC = 0x8014


## Remainder Signed: rem

* "rem" divides the content of two CPU registers (could be the same CPU register twice) and save the reminder result of the division into a CPU register (could be the same as either operands). The content of the two CPU registers is going to be interpreted as signed integers.
  * Be careful with possible exception: divide by zero, again.

* Example:
  ```
  .text
  main:
    li      a0, 5
    li      a1, 2
    rem     a2, a0, a1   # a2 = 5 % 2
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:rem1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 5,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x5; register PC = 0x8008
2,pc = 0x8008,li a1 2,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x2; register PC = 0x8010
4,pc = 0x8010,rem a2 a0 a1,register R12 = 0x1; register PC = 0x8014


# Logic instructions

## Or: or, ori

* Example:
  ```
  .text
  main:
    li      a0, 0xF875
    li      a1, 0x00FF
    or      a2, a0, a1    # a2 = 0xF875 | 0x00FF
    ori     a3, a0, 0xFF  # a2 = 0xF875 | 0x00FF
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:or1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 0xF875,register R10 = 0xf000; register PC = 0x8004
1,pc = 0x8004,,register R10 = 0xf875; register PC = 0x8008
2,pc = 0x8008,li a1 0x00FF,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0xff; register PC = 0x8010
4,pc = 0x8010,or a2 a0 a1,register R12 = 0xf8ff; register PC = 0x8014
5,pc = 0x8014,ori a3 a0 0xFF,register R13 = 0xf8ff; register PC = 0x8018


## And: and, andi

* Example:
  ```
  .text
  main:
    li      a0, 0xF875
    li      a1, 0x00FF
    and     a2, a0, a1    # a2 = 0xF875 & 0x00FF
    andi    a3, a0, 0xFF  # a2 = 0xF875 & 0x00FF
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:and1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 0xF875,register R10 = 0xf000; register PC = 0x8004
1,pc = 0x8004,,register R10 = 0xf875; register PC = 0x8008
2,pc = 0x8008,li a1 0x00FF,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0xff; register PC = 0x8010
4,pc = 0x8010,and a2 a0 a1,register R12 = 0x75; register PC = 0x8014
5,pc = 0x8014,andi a3 a0 0xFF,register R13 = 0xf974; register PC = 0x8018


## Xor: xor, xori

* Example:
  ```
  .text
  main:
    li      a0, 0xF875
    li      a1, 0x00FF
    xor     a2, a0, a1    # a2 = 0xF875 ^ 0x00FF
    xori    a3, a0, 0xFF  # a2 = 0xF875 ^ 0x00FF
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:xor1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 0xF875,register R10 = 0xf000; register PC = 0x8004
1,pc = 0x8004,,register R10 = 0xf875; register PC = 0x8008
2,pc = 0x8008,li a1 0x00FF,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0xff; register PC = 0x8010
4,pc = 0x8010,xor a2 a0 a1,register R12 = 0xf88a; register PC = 0x8014
5,pc = 0x8014,xori a3 a0 0xFF,register R13 = 0xf88a; register PC = 0x8018


## Shift Right Logical: srl, srli

* Example:
  ```
  .text
  main:
    li      a0, 0x010
    li      a1, 0x3
    srl     a2, a0, a1   # a2 = 0x010 >> 3
    srli    a2, a0, 3    # a2 = 0x010 >> 3
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:srl1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')

if try_in_wepsim:
  show_asm_in_ws('/tmp/example.asm','RISCV')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 0x010,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x10; register PC = 0x8008
2,pc = 0x8008,li a1 0x3,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x3; register PC = 0x8010
4,pc = 0x8010,srl a2 a0 a1,register R12 = 0x2; register PC = 0x8014
5,pc = 0x8014,srli a2 a0 3,register PC = 0x8018


# Shift instructions

## Shift Left Logical: sll, slli

* Example:
  ```
  .text
  main:
    li      a0, 0x010
    li      a1, 0x3
    sll     a2, a0, a1   # a2 = 0x010 << 3
    slli    a2, a0, 3    # a2 = 0x010 << 3
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:sll1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 0x010,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x10; register PC = 0x8008
2,pc = 0x8008,li a1 0x3,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x3; register PC = 0x8010
4,pc = 0x8010,sll a2 a0 a1,register R12 = 0x80; register PC = 0x8014
5,pc = 0x8014,slli a2 a0 3,register PC = 0x8018


## Shift Right Arithmetic: sra, srai

* Example:
  ```
  .text
  main:
    li      a0, 0x010
    li      a1, 0x3
    sra     a2, a0, a1   # a2 = 0x010 >> 3
    srai    a2, a0, 3    # a2 = 0x010 >> 3
  ```

In [ ]:
#@markdown <font size='4'>Run the cell above</font>
try_in_wepsim = False # @param {type:"boolean"}

search_term = 'id:sra1'
write_cell_above_to_file(search_term, '/tmp/example.asm')

!./wepsim/wepsim.sh -a stepbystep -m ep -f ./wepsim/repo/microcode/rv32/ep_base.mc -s /tmp/example.asm > /tmp/result.csv

status, df1 = show_results('/tmp/result.csv')


,pc,instruction,changes_from_zero_or_current_value
0,pc = 0x8000,li a0 0x010,register PC = 0x8004
1,pc = 0x8004,,register R10 = 0x10; register PC = 0x8008
2,pc = 0x8008,li a1 0x3,register PC = 0x800c
3,pc = 0x800c,,register R11 = 0x3; register PC = 0x8010
4,pc = 0x8010,sra a2 a0 a1,register R12 = 0x2; register PC = 0x8014
5,pc = 0x8014,srai a2 a0 3,register PC = 0x8018
